In [2]:
import pandas as pd
import numpy as np

In [28]:
from olist.product import Product
from olist.seller import Seller
from olist.order import Order
from olist.data import Olist

# Product Cat

In [5]:
product = Product()

products = product.get_training_data()
products.head(5)

In [12]:
columns = list(products.select_dtypes(exclude=['object']).columns)

In [16]:
agg = "mean"
agg_params = dict(zip(columns, [agg] * len(columns)))

In [17]:
agg_params['quantity'] = 'sum'

In [19]:
agg_params

{'product_name_length': 'mean',
 'product_description_length': 'mean',
 'product_photos_qty': 'mean',
 'product_weight_g': 'mean',
 'product_length_cm': 'mean',
 'product_height_cm': 'mean',
 'product_width_cm': 'mean',
 'wait_time': 'mean',
 'price': 'mean',
 'share_of_one_stars': 'mean',
 'share_of_five_stars': 'mean',
 'review_score': 'mean',
 'n_orders': 'mean',
 'quantity': 'sum',
 'sales': 'mean'}

In [20]:
product_cat = products.groupby('quantity').agg(agg_params)

# Seller - Cost of Review

In [29]:
order = Order()
data = Olist()

In [25]:
order_reviews = order.get_review_score()

In [31]:
order_sellers = data.get_data()['order_items'][['order_id', 'seller_id']].drop_duplicates()

In [33]:
order_reviews

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [34]:
df = order_sellers.merge(order_reviews, on='order_id')

In [41]:
df['cost_of_review'] = df.review_score.map({1: 100,
                                            2: 50,
                                            3: 40,
                                            4: 0,
                                            5: 0})

In [44]:
result = df.groupby('seller_id', as_index=False).agg(
            {'dim_is_five_star': 'mean',
            'dim_is_one_star': 'mean',
            'review_score': 'mean',
            'cost_of_review': 'sum'})

result.columns = ['seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score', 'cost_of_review']

In [45]:
result

,seller_id,share_of_five_stars,share_of_one_stars,review_score,cost_of_review
0,0015a82c2db000af6aaaf3ae2ecb0532,0.666667,0.333333,3.666667,100
1,001cca7ae9ae17fb1caed9dfb1094831,0.527919,0.121827,3.984772,3560
2,001e6ad469a905060d959994f1b41e4f,0.000000,1.000000,1.000000,100
3,002100f778ceb8431b7a1020ff7ab48f,0.519231,0.134615,3.903846,1110
4,003554e2dce176b5555353e4f3555ac8,1.000000,0.000000,5.000000,0
...,...,...,...,...,...
3085,ffcfefa19b08742c5d315f2791395ee5,0.000000,1.000000,1.000000,100
3086,ffdd9f82b9a447f6f8d4b91554cc7dd3,0.666667,0.055556,4.333333,190
3087,ffeee66ac5d5a62fe688b9d26f83f534,0.642857,0.142857,4.214286,200
3088,fffd5413c0700ac820c7069d66d98c89,0.542373,0.186441,3.847458,1360


# Seller - IT costs